![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
# Chapter 8: Single Degree of Freedom (SDoF) Fit Method

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

# Imports

In [ ]:
from bladesight import Datasets
from bladesight.btt import get_rotor_blade_AoAs, get_blade_tip_deflections_from_AoAs
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from typing import List, Tuple, Dict

from scipy.optimize import differential_evolution

# Getting the dataset


In [ ]:
ds = Datasets["data/intro_to_btt/intro_to_btt_ch05"]
df_opr_zero_crossings = ds['table/opr_zero_crossings']
df_prox_1 = ds['table/prox_1_toas']
df_prox_2 = ds['table/prox_2_toas']
df_prox_3 = ds['table/prox_3_toas']
df_prox_4 = ds['table/prox_4_toas']

BLADE_COUNT = 5
RADIUS = 164000

rotor_blade_AoA_dfs = get_rotor_blade_AoAs(
    df_opr_zero_crossings,
    [df_prox_1, df_prox_2, df_prox_3, df_prox_4],
    np.cumsum(np.deg2rad(np.array([19.34, 19.34, 19.34]))), 
    BLADE_COUNT
)
tip_deflection_dfs = []
for df_AoAs in rotor_blade_AoA_dfs:
    df_tip_deflections = get_blade_tip_deflections_from_AoAs(
        df_AoAs,
        RADIUS,
        11,
        2,
        0.5
    )
    tip_deflection_dfs.append(df_tip_deflections)


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=tip_deflection_dfs[0]['n'],
        y=tip_deflection_dfs[0]["pk-pk"],
        name='Pk-Pk (Blade 1)',
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        x=tip_deflection_dfs[0]['n'],
        y=tip_deflection_dfs[0]["Omega"] / (2*np.pi) * 60,
        name='Shaft speed',
    ),
    secondary_y=False,
)
fig.update_layout(
    title='Blade Tip Deflection',
    xaxis_title='Shaft revolution',
    yaxis_title='Peak to Peak vibration [µm]',
    yaxis2_title='Shaft speed [RPM]',
)

fig.show()

In [ ]:
df_up = tip_deflection_dfs[0].query('n < 1450')
df_down = tip_deflection_dfs[0].query('n > 1450')

In [ ]:
fig = go.Figure()

df_up = tip_deflection_dfs[0].query('n < 1450')
df_down = tip_deflection_dfs[0].query('n > 1450')
fig.add_trace(
    go.Scatter(
        x=df_up["Omega"] * 60 / (2*np.pi),
        y=df_up["pk-pk"],
        name='pk-pk (Blade 1 run-up)',
    )
)

fig.add_trace(
    go.Scatter(
        x=df_down["Omega"] * 60 / (2*np.pi),
        y=df_down["pk-pk"],
        name='pk-pk (Blade 1 run-down)',
    )
)


fig.update_layout(
    title='Blade Tip Deflection run-up vs run-down',
    xaxis_title='Shaft speed [RPM]',
    yaxis_title='Peak to Peak vibration [µm]'
)

fig.show()

In [ ]:
print("RPM diff at EO=13", 572.5-581.95   )
print("RPM diff at EO=8", 934.05-950.52   )
print("RPM diff at EO=7", 1068.4 - 1084   )
print("RPM diff at EO=6", 1232.14-1274.14   )

In [ ]:
572 / 60 * 13

## Guessing the EO

In [ ]:
EO = 125 / (950/60)
print("The EO is", EO, "~", round(EO))
EO = round(EO)

## Resonance window selection


In [ ]:
df_resonance_window = tip_deflection_dfs[0].query("n > 500 and n < 600")

## Creating a simple objective function


In [ ]:
def get_X(
        omega : np.ndarray,
        omega_n : float, 
        zeta: float,
        delta_st: float
    ) -> np.ndarray:
    """
    This function returns the vibration amplitude of 
    the blade vibration.
    
    `x(ω) = 	δ_st / sqrt( (1 - r**2)**2 + (2*ζ*r)**2)`

    where:

    r = ω/ω_0

    Args:

        omega (np.ndarray): The excitation frequencies in rad/s.
        
        omega_n (float): The natural frequency of the blade in rad/s.
        
        zeta (float): The damping ratio of the blade vibration.
        
        delta_st (float, optional): The static deflection of the blade. 
            This value is usually given in units of µm.

    Returns:

        np.ndarray: The amplitude of the blade vibration in the
            same units as delta_st.
    """
    r = omega / omega_n
    return (
        delta_st 
        / np.sqrt(
            (1 - r**2)**2 
            + (2*zeta*r)**2
        )
    )

def get_phi(
    omega : np.ndarray, 
    omega_n : float, 
    zeta: float
) -> np.ndarray:
    """
    Get the phase between the tip deflection and 
        the forcing function. 

    φ(ω) = arctan(2*ζ*r /  (1 - r**2))
    
    where:
    r = ω/ω_n

    Args:
        omega (np.ndarray): The excitation frequencies in rad/s.

        omega_0 (float): The natural frequency of the blade in rad/s.
        
        delta (float): The damping ratio of the blade vibration.

    Returns:

        np.ndarray: The phase of the blade vibration in rad.
    """
    r = omega / omega_n
    return np.arctan2(2 * zeta * r,1 - r**2)

def predict_sdof_samples_simple(
    omega_n : float,
    zeta : float,
    delta_st : float,
    EO : int,
    theta_sensor : float,
    arr_omega : np.ndarray
) -> np.ndarray:
    """
    This function determined the predicted SDoF fit
    samples at a proximity probe given the SDoF parameters.

    Args:

        omega_n (float): The natural frequency of the SDoF system.
        
        zeta (float): The damping ratio of the SDoF system.
        
        delta_st (float): The static deflection of the SDoF system.
        
        EO (int): The EO of vibration you want to fit.
        
        theta_sensor (float): The sensor's angular position on the rotor.
        
        arr_omega (np.ndarray): The angular velocity of the rotor corresponding
            to each revolution for which we want to predict the SDoF samples.

    Returns:

        np.ndarray: The predicted SDoF samples.
    """
    X = get_X(arr_omega*EO, omega_n, zeta, delta_st)
    phi = get_phi(arr_omega*EO, omega_n, zeta)
    predicted_tip_deflections = X * np.cos(theta_sensor * EO - phi)
    return predicted_tip_deflections

def SDoF_simple_loss(
        model_params : np.ndarray,
        arr_tip_deflections : np.ndarray,
        arr_omega : np.ndarray, 
        EO : int, 
        theta_sensor : float
    ) -> np.ndarray:
    """
    This function returns the error between the SDoF model given the 
    parameters and the measured tip deflection data.

    Args:
        
        model_params (np.ndarray): The SDoF fit method's model parameters. It
            includes a list of the following parameters:
            omega_n (float): The natural frequency of the SDoF system.
            ln_zeta (float): The damping ratio of the SDoF system.
            delta_st (float): The static deflection of the SDoF system.
        
        arr_tip_deflections (np.ndarray): The tip deflection data of the probe.
        
        arr_omega (np.ndarray): The angular velocity of the rotor corresponding
            to the tip deflection data.
        
        EO (int): The EO of vibration you want to fit.
        
        theta_sensor (float): The sensor's angular position on the rotor.

    Returns:

        np.ndarray: The sum of squared error between the tip deflection data
            and the predicted tip deflections.
    """
    omega_n, ln_zeta, delta_st = model_params
    zeta = np.exp(ln_zeta)
    predicted_tip_deflections = predict_sdof_samples_simple(
        omega_n, zeta, delta_st, EO, theta_sensor, arr_omega
    )
    return np.sum(
        (
            arr_tip_deflections
            - predicted_tip_deflections
        )**2
    )


## Optimization bounds

In [ ]:
omega_n_min = df_resonance_window["Omega"].min() * EO
omega_n_max = df_resonance_window["Omega"].max() * EO
ln_zeta_min = np.log(0.0001)
ln_zeta_max = np.log(0.3)
delta_st_min = 0
delta_st_max = 10

## Solving the simple problem

In [ ]:
simple_solution = differential_evolution(
    func = SDoF_simple_loss,
    bounds=[
        (omega_n_min, omega_n_max),
        (ln_zeta_min, ln_zeta_max),
        (delta_st_min, delta_st_max)
    ],
    args=(
        df_resonance_window[f'x_p1_filt'].values,
        df_resonance_window['Omega'].values,
        EO,
        df_resonance_window["AoA_p1"].median()
    ),
    seed=42
)


In [ ]:
print("ω_n = ", simple_solution.x[0] / (2*np.pi), " Hz")
print("ζ   = ", np.exp(simple_solution.x[1]))
print("δ_st= ", simple_solution.x[2], " µm")

In [ ]:
predicted_tip_deflections = predict_sdof_samples_simple(
    simple_solution.x[0],
    np.exp(simple_solution.x[1]),
    simple_solution.x[2],
    EO,
    df_resonance_window["AoA_p1"].median(),
    df_resonance_window['Omega'].values
)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=df_resonance_window[f'x_p1_filt'],
        name='Tip deflection',
        mode='markers+lines'
    )
)
fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=predicted_tip_deflections,
        name='SDoF fit',
        mode='markers+lines'
    )
)

fig.update_layout(
    title=f"Tip deflection vs. SDoF fit",
    xaxis_title="Revolution number",
    yaxis_title="Tip deflection [µm]",
)

fig.show()




## Adding amplitude and phase offsets


In [ ]:
phi_0_min = 0
phi_0_max = 2*np.pi
z_max = df_resonance_window["x_p1_filt"].abs().max()
z_min = -z_max


In [ ]:
def predict_sdof_samples(
    omega_n : float,
    zeta : float,
    delta_st : float,
    EO : int,
    theta_sensor : float,
    phi_0 : float,
    arr_omega : np.ndarray
) -> np.ndarray:
    """
    This function determined the predicted SDoF fit
    samples at a proximity probe given the SDoF parameters.

    Args:

        omega_n (float): The natural frequency of the SDoF system.
        
        zeta (float): The damping ratio of the SDoF system.
        
        delta_st (float): The static deflection of the SDoF system.
        
        phi_0 (float): The phase offset of the SDoF system.
        
        EO (int): The EO of vibration you want to fit.
        
        theta_sensor (float): The sensor's angular position on the rotor.
        
        phi_0 (float): The phase offset of the SDoF system.
        
        arr_omega (np.ndarray): The angular velocity of the rotor corresponding
            to each revolution for which we want to predict the SDoF samples.

    Returns:
        
        np.ndarray: The predicted SDoF samples.
    """
    X = get_X(arr_omega*EO, omega_n, zeta, delta_st)  
    phi = get_phi(arr_omega*EO, omega_n, zeta)
    predicted_tip_deflections = X * np.cos(theta_sensor * EO - phi + phi_0)
    return predicted_tip_deflections

def get_correction_values(
    arr_omegas : float,
    z_median : float,
    z_max : float, 
) -> np.ndarray:
    """
    This function calculates the correction values for each sample
    based on the correction factors.

    Args:
        arr_omegas (float): The omega values for each sample.
        
        z_median (float): The correction value at the median shaft speed.
        
        z_max (float): The correction value at the max shaft speed.

    Returns:
        np.ndarray: The sample offsets for each sample.
    """
    omega_median = np.median(arr_omegas)
    omega_max = np.min(arr_omegas)
    m = (
        z_max
        - z_median
    ) / (
        omega_max 
        - omega_median
    )
    b = z_median - m * omega_median
    correction_values = m * arr_omegas  + b
    return correction_values

def SDoF_loss(
        model_params : np.ndarray,
        arr_tip_deflections : np.ndarray,
        arr_omega : np.ndarray, 
        EO : int, 
        theta_sensor : float
) -> np.ndarray:
    """
    This function fits the SDoF parameters to a single 
    probe's tip deflection data.

    Args:
        model_params (np.ndarray): The SDoF fit method's model parameters. It
            includes a list of the following parameters:
            omega_n (float): The natural frequency of the SDoF system.
            ln_zeta (float): The damping ratio of the SDoF system.
            delta_st (float): The static deflection of the SDoF system.
            phi_0 (float): The phase offset of the SDoF system.
            z_median (float): The amplitude offset at the median shaft speed.
            z_max (float): The maximum amplitude offset.

        arr_tip_deflections (np.ndarray): The tip deflection data of the probe.
        
        arr_omega (np.ndarray): The angular velocity of the rotor corresponding
            to the tip deflection data.
        
        EO (int): The EO of vibration you want to fit.
        
        theta_sensor (float): The sensor's angular position on the rotor.

    Returns:
        np.ndarray: The sum of squared error between the tip deflection data
    """
    omega_n, ln_zeta, delta_st, phi_0, z_median, z_max = model_params
    zeta = np.exp(ln_zeta)
    predicted_tip_deflections = predict_sdof_samples(
        omega_n, zeta, delta_st, EO, theta_sensor, phi_0, arr_omega
    )
    arr_tip_deflection_corrections = get_correction_values(
        arr_omega, z_median, z_max
    )
    arr_tip_deflections_corrected = (
        arr_tip_deflections
        + arr_tip_deflection_corrections
    )
    return np.sum(
        (
            arr_tip_deflections_corrected
            - predicted_tip_deflections
        )**2
    )


## Solving the improved problem


In [ ]:
improved_solution = differential_evolution(
    func = SDoF_loss,
    bounds=[
        (omega_n_min, omega_n_max),
        (ln_zeta_min, ln_zeta_max),
        (delta_st_min, delta_st_max),
        (phi_0_min, phi_0_max),
        (z_min, z_max),
        (z_min, z_max),
    ],
    args=(
        df_resonance_window[f'x_p1_filt'].values,
        df_resonance_window['Omega'].values,
        EO,
        df_resonance_window["AoA_p1"].median()
    ),
    seed=42
)


In [ ]:
print("ω_n      = ", improved_solution.x[0] / (2*np.pi), " Hz")
print("ζ        = ", np.exp(improved_solution.x[1]))
print("δ_st     = ", improved_solution.x[2], " µm")
print("φ_0      = ", improved_solution.x[3], " rad")
print("z_median = ", improved_solution.x[4], " µm")
print("z_max    = ", improved_solution.x[5], " µm")


In [ ]:
predicted_tip_deflections = predict_sdof_samples(
    improved_solution.x[0],
    np.exp(improved_solution.x[1]),
    improved_solution.x[2],
    EO,
    df_resonance_window["AoA_p1"].median(),
    improved_solution.x[3],
    df_resonance_window['Omega'].values
)
corrected_tip_deflections = (
    df_resonance_window[f'x_p1_filt'].values
    + get_correction_values(
        df_resonance_window['Omega'].values,
        improved_solution.x[4],
        improved_solution.x[5]
    )
)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=corrected_tip_deflections,
        name='Corrected tip deflections',
        mode='markers+lines'
    )
)
fig.add_trace(
    go.Scatter(
        x=df_resonance_window['n'],
        y=predicted_tip_deflections,
        name='SDoF fit',
        mode='markers+lines'
    )
)
fig.update_layout(
    title=f"Tip deflection vs. SDoF fit",
    xaxis_title="Revolution number",
    yaxis_title="Tip deflection [µm]",
)

fig.show()


## Multiple probe solution


In [ ]:
def SDoF_loss_multiple_probes(
        model_params : np.ndarray,
        tip_deflections_set : List[np.ndarray],
        arr_omega : np.ndarray, 
        EO : int, 
        theta_sensor_set : List[float]
) -> np.ndarray:
    omega_n, ln_zeta, delta_st, phi_0, *correction_factors = model_params
    zeta = np.exp(ln_zeta)
    error = 0
    for i_probe, arr_tip_deflections in enumerate(tip_deflections_set):
        theta_sensor = theta_sensor_set[i_probe]
        predicted_tip_deflections = predict_sdof_samples(
            omega_n, zeta, delta_st, EO, theta_sensor, phi_0, arr_omega
        )
        z_median = correction_factors[i_probe*2]
        z_max = correction_factors[i_probe*2+1]
        arr_tip_deflection_corrections = get_correction_values(
            arr_omega, z_median, z_max
        )
        arr_tip_deflections_corrected = (
            arr_tip_deflections
            + arr_tip_deflection_corrections
        )
        error += np.sum(
            (
                arr_tip_deflections_corrected
                - predicted_tip_deflections
            )**2
        )
    return error


In [ ]:
PROBE_COUNT = 4
bounds = [
    (omega_n_min, omega_n_max),
    (ln_zeta_min, ln_zeta_max),
    (delta_st_min, delta_st_max),
    (phi_0_min, phi_0_max),
]
tip_deflections_set = []
theta_sensor_set = []
for i_probe in range(PROBE_COUNT):
    z_max = df_resonance_window[f"x_p{i_probe+1}_filt"].abs().max()
    z_min = -z_max
    bounds.extend(
        [
            (z_min, z_max),
            (z_min, z_max)
        ]
    )
    tip_deflections_set.append(
        df_resonance_window[f"x_p{i_probe+1}_filt"].values
    )
    theta_sensor_set.append(
        df_resonance_window[f"AoA_p{i_probe+1}"].median()
    )

multiple_probes_solution = differential_evolution(
    func = SDoF_loss_multiple_probes,
    bounds=bounds,
    args=(
        tip_deflections_set,
        df_resonance_window['Omega'].values,
        EO,
        theta_sensor_set
    ),
    seed=42
)


In [ ]:
print("ω_n      = ", multiple_probes_solution.x[0] / (2*np.pi), " Hz")
print("ζ        = ", np.exp(multiple_probes_solution.x[1]))
print("δ_st     = ", multiple_probes_solution.x[2], " µm")
print("φ_0      = ", multiple_probes_solution.x[3], " rad")
for i_probe in range(PROBE_COUNT):
    print(f"z_median_{i_probe+1} = ", multiple_probes_solution.x[4+i_probe*2], " µm")
    print(f"z_max_{i_probe+1}    = ", multiple_probes_solution.x[5+i_probe*2], " µm")


In [ ]:
for i_probe in range(PROBE_COUNT):
    predicted_tip_deflections = predict_sdof_samples(
        multiple_probes_solution.x[0],
        np.exp(multiple_probes_solution.x[1]),
        multiple_probes_solution.x[2],
        EO,
        theta_sensor_set[i_probe],
        multiple_probes_solution.x[3],
        df_resonance_window['Omega'].values
    )
    corrected_tip_deflections = (
        df_resonance_window[f'x_p{i_probe+1}_filt'].values
        + get_correction_values(
            df_resonance_window['Omega'].values,
            multiple_probes_solution.x[4+i_probe*2],
            multiple_probes_solution.x[5+i_probe*2]
        )
    )
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df_resonance_window['n'],
            y=corrected_tip_deflections,
            name='Corrected tip deflections',
            mode='markers+lines'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_resonance_window['n'],
            y=predicted_tip_deflections,
            name='SDoF fit',
            mode='markers+lines'
        )
    )
    fig.update_layout(
        title=f"Tip deflection vs. SDoF fit, probe {i_probe + 1}",
        xaxis_title="Revolution number",
        yaxis_title="Tip deflection [µm]",
    )
    fig.show()

## Guessing the EO


In [ ]:
PROBE_COUNT = 4
EOs = np.arange(1,17)
EO_solutions = []

for EO in EOs:
    print("NOW SOLVING FOR EO = ", EO)
    omega_n_min = df_resonance_window["Omega"].min() * EO
    omega_n_max = df_resonance_window["Omega"].max() * EO
    ln_zeta_min = np.log(0.0001)
    ln_zeta_max = np.log(0.3)
    delta_st_min = 0
    delta_st_max = 10
    phi_0_min = 0
    phi_0_max = 2*np.pi
    bounds = [
        (omega_n_min, omega_n_max),
        (ln_zeta_min, ln_zeta_max),
        (delta_st_min, delta_st_max),
        (phi_0_min, phi_0_max),
    ]
    tip_deflections_set = []
    theta_sensor_set = []
    for i_probe in range(PROBE_COUNT):
        z_max = df_resonance_window[f"x_p{i_probe+1}_filt"].abs().max()
        z_min = -z_max
        bounds.extend(
            [
                (z_min, z_max),
                (z_min, z_max)
            ]
        )
        tip_deflections_set.append(
            df_resonance_window[f"x_p{i_probe+1}_filt"].values
        )
        theta_sensor_set.append(
            df_resonance_window[f"AoA_p{i_probe+1}"].median()
        )

    multiple_probes_solution = differential_evolution(
        func = SDoF_loss_multiple_probes,
        bounds=bounds,
        args=(
            tip_deflections_set,
            df_resonance_window['Omega'].values,
            EO,
            theta_sensor_set
        ),
        seed=42
    )
    EO_solutions.append(multiple_probes_solution)


In [ ]:
fig = go.Figure()

EO_errors = [
    EO_solution.fun
    for EO_solution in EO_solutions
]

fig.add_trace(
    go.Bar(
        x=EOs,
        y=EO_errors,
        name='EO Error',
    )
)
fig.update_layout(
    title='EO Error',
    xaxis_title='EO',
    yaxis_title='Error',
)

fig.show()

## Coding exercises

### 1. Getting the amplitude larger


In [ ]:
def SDoF_loss_multiple_probes(
        model_params : np.ndarray,
        tip_deflections_set : List[np.ndarray],
        arr_omega : np.ndarray, 
        EO : int, 
        theta_sensor_set : List[float],
        amplitude_scaling_factor : float = 1 
    ) -> np.ndarray:
    ...
    # Please complete me!

### 2. Writing a function we can use

In [ ]:

def perform_SDoF_fit(
    df_blade : pd.DataFrame,
    n_start : int,
    n_end : int,
    EOs : List[int] = np.arange(1, 20),
    delta_st_max : int = 10,
    verbose : bool = False
) -> Dict[str, float]:
    ...
    # Please complete me!

### 3. Comparing resonances on the upward and downward ramps

In [ ]:
# Compare the results of the ramp-up and ramp-down